In [ ]:
import torch
import numpy as np
from PIL import Image
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.nn import functional as F
from sklearn.utils import resample
from torchvision.transforms import ToTensor
from torchvision import datasets, models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GroupKFold 

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

In [ ]:
# paths to patch directories 

patch_size = {patch_size}

patch_dir_tnbc = 'PATH-TO-IMPRESS-TNBC-PATCH-DIR'
patch_dir_her2 = 'PATH-TO-IMPRESS-HER2-PATCH-DIR'

In [ ]:
# paths to patch embeddings 

her2_feats_path = 'PATH-TO-IMPRESS-TNBC-PATCH-EMBEDDINGS-FILE(.pth)' 
tnbc_feats_path = 'PATH-TO-IMPRESS-HER2-PATCH-EMBEDDINGS-FILE(.pth)'

**LOADING PATCH DATA**

In [ ]:
# reading patch info, and obtaining patch paths and labels 

her2_df = pd.read_csv('PATH-TO-PATCH-INFO-DATA-CSV')
tnbc_df = pd.read_csv('PATH-TO-PATCH-INFO-DATA-CSV')

def obtain_patch_path(row, patch_dir): 
    
    wsi = str(row['WSI No.'])
    x = str(row['X Pixel'])
    y = str(row['Y Pixel'])
    
    if len(wsi) == 2: 
        return f'{patch_dir}/0{wsi}_{x}_{y}.jpg'
    else: 
        return f'{patch_dir}/{wsi}_{x}_{y}.jpg'

her2_df['Patch Path'] = her2_df.apply(lambda row: obtain_patch_path(row, patch_dir_her2), axis=1)
tnbc_df['Patch Path'] = tnbc_df.apply(lambda row: obtain_patch_path(row, patch_dir_tnbc), axis=1)

# patch labels 
her2_df['Label'] = [1] * len(her2_df) 
tnbc_df['Label'] = [0] * len(tnbc_df) 

In [ ]:
tnbc_df = tnbc_df[tnbc_df['Annotation Id'] != -1]
her2_df = her2_df[her2_df['Annotation Id'] != -1]

tnbc_df = tnbc_df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"])
her2_df = her2_df.sort_values(by=["WSI No.", "Annotation Id", "Y Pixel", "X Pixel"])

In [ ]:
print(len(tnbc_df), len(her2_df))

**DATASET SPLIT** 

In [ ]:
train_her2_df = her2_df[her2_df['WSI No.'] < 101] 
val_her2_df = her2_df[(her2_df['WSI No.'] >= 101) & (her2_df['WSI No.'] < 111)] 
test_her2_df = her2_df[her2_df['WSI No.'] >= 111] 

train_tnbc_df = tnbc_df[tnbc_df['WSI No.'] < 942] 
val_tnbc_df = tnbc_df[(tnbc_df['WSI No.'] >= 942) & (tnbc_df['WSI No.'] < 952)] 
test_tnbc_df = tnbc_df[tnbc_df['WSI No.'] >= 952] 

train_df = pd.concat([train_her2_df, train_tnbc_df]) 
val_df = pd.concat([val_her2_df, val_tnbc_df]) 
test_df = pd.concat([test_her2_df, test_tnbc_df]) 

In [ ]:
train_wsi_nos = train_df['WSI No.'].to_list() 
train_ann_nos = train_df['Annotation Id'].to_list() 
train_labels_ = train_df['Label'].to_list() 
train_patch_paths = train_df['Patch Path'].to_list() 

val_wsi_nos = val_df['WSI No.'].to_list() 
val_ann_nos = val_df['Annotation Id'].to_list() 
val_labels_ = val_df['Label'].to_list() 
val_patch_paths = val_df['Patch Path'].to_list() 

test_wsi_nos = test_df['WSI No.'].to_list() 
test_ann_nos = test_df['Annotation Id'].to_list() 
test_labels_ = test_df['Label'].to_list() 
test_patch_paths = test_df['Patch Path'].to_list() 

In [ ]:
patch_paths = train_patch_paths + val_patch_paths + test_patch_paths 
patch_labels = train_patch_labels + val_patch_labels + test_patch_labels 
wsi_ids = train_wsi_nos + val_wsi_nos + test_wsi_nos 

**CLASS DISTRIBUTION** 

In [ ]:
def plot_bag_labels(ax, title, labels):
    ax.bar(['Negative', 'Positive'], [labels.count(0), labels.count(1)])
    ax.set_title(title)
    ax.set_ylabel('Count')

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

plot_bag_labels(axs[0], 'TRAIN LABELS', train_patch_labels)
plot_bag_labels(axs[1], 'VALIDATION LABELS', val_patch_labels)
plot_bag_labels(axs[2], 'TEST LABELS', test_patch_labels)

plt.tight_layout()
plt.show() 

**CUSTOM DATASETS & DATA LOADERS** 

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, patches, labels):
        assert len(patches) == len(labels), "Mismatch in number of patches and labels"
        self.patches = patches
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Check for valid index range
        if idx >= len(self.labels) or idx >= len(self.patches):
            raise IndexError(f"Index {idx} out of range for dataset with length {len(self)}")

        patch = Image.open(self.patches[idx]).convert("RGB")  # Adjust this if `patches` are file paths
        patch = patch.resize((224, 224))  
        patch = ToTensor()(patch)
        patch = nn.functional.normalize(patch, dim=0, p=2)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return patch, label 

**FINAL MODEL** 

In [ ]:
# defining final pipeline to process prepared bags 

class Pipeline(nn.Module):
    def __init__(self,num_classes):
        #define necessary layers
        super().__init__()
        self.num_classes = num_classes
          
        self.base = models.{'MODEL_NAME'}(weights=True)
        
        # Unfreeze model weights
        for param in self.base.parameters():
            param.requires_grad = True
        
        self.flatten = nn.Flatten()
        
        self.head = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        ) 
    
    def forward(self,X):
        X = self.base(X)
        X = self.flatten(X)
        X = self.head(X)
        return X, F.sigmoid(X)

In [ ]:
# Setting some hyperparameters

batch_size = {BATCH_SIZE}
num_epochs = {NUMBER-OF-EPOCHS}
num_folds = {NUM_FOLDS}
num_of_workers = {NUMBER-OF-WORKERS}
group_kfold = GroupKFold(n_splits=num_folds)
metrics = {
    fold + 1: {
        'train_loss': [], 'train_accuracy': [], 'train_roc_auc': [],
        'valid_loss': [], 'valid_accuracy': [], 'valid_roc_auc': [],
        'wsi_nos': [], 'y_pred':[], 'y_true':[]
    } for fold in range(num_folds)
}

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

**5 FOLD CROSS VALIDATION** 

In [ ]:
# Accumulate all fold results 

all_y_true = []
all_y_scores = []
all_y_pred = []

In [ ]:
# K Fold cross validation loop

for fold, (train_idx, test_idx) in enumerate(group_kfold.split(patch_paths, patch_labels, groups=wsi_ids)):
    
    # Splitting bags and labels for each fold without converting to numpy array
    train_patch_paths = [patch_paths[i] for i in train_idx]
    test_patch_paths = [patch_paths[i] for i in test_idx]
    train_patch_labels = [patch_labels[i] for i in train_idx]
    test_patch_labels = [patch_labels[i] for i in test_idx]
    
    metrics[fold+1]['wsi_nos'] = [wsi_ids[i] for i in test_idx] 
    
    # Creating datasets and loaders for each fold
    train_dataset = CustomDataset(train_patch_paths, train_patch_labels)
    val_dataset = CustomDataset(test_patch_paths, test_patch_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 16)
    valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers = 16)

    # Output the fold details
    print(f"\nFold {fold + 1}")
    print(f"Train WSI IDs: {set(np.array(wsi_ids)[train_idx])}")
    print(f"Test WSI IDs: {set(np.array(wsi_ids)[test_idx])}")
    print(f"Train size: {len(train_loader.dataset)}, Test size: {len(valid_loader.dataset)}") 

    # Initialize empty lists to store loss and accuracy for training and validation
    train_losses = []
    valid_losses = []
    train_accuracies = []
    valid_accuracies = []
    roc_values_train = []
    roc_values_val = []
    
    # modifying first layer for one color channel 
    pos_weight = torch.tensor([POSITIVE-WEIGHT])
    pos_weight = pos_weight.to(device)
    loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
    model = Pipeline(1)

    # Check if GPU is available
    if torch.cuda.is_available():
        model = model.to(device)
        print('available')

    #criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=model.parameters(), lr={LEARNING_RATE}) 

    for epoch in range(num_epochs):
        model.train()
        y_true_train = []
        y_scores_train = []
        train_loss = 0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs_without_sigmoid, outputs_with_sigmoid = model(images)
            loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()  # L+=l.item()
            predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            y_true_train.extend(labels.cpu().detach().numpy())
            y_scores_train.extend(outputs_with_sigmoid.cpu().detach().numpy())
        
        # End of training loop
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(100 * correct / total)
        
        # roc auc logic
        fpr_train, tpr_train, _ = roc_curve(y_true_train, y_scores_train)
        roc_auc_train = auc(fpr_train, tpr_train)
        roc_values_train.append(roc_auc_train)

        # Plot ROC curve for the training set

        # Validate your model after each epoch if needed
        model.eval()
        valid_loss = 0
        correct = 0
        total = 0
        y_true_val = []
        y_scores_val = []
        with torch.no_grad():
            for images, labels in tqdm(valid_loader):
                # Move data to GPU if available
                images = images.to(device)
                labels = labels.to(device)
                outputs_without_sigmoid, outputs_with_sigmoid = model(images)
                loss = loss_fn(outputs_without_sigmoid.squeeze(-1), labels.float())
                valid_loss += loss.item()  # L+=l.item()
                predicted = (outputs_with_sigmoid >= 0.5).long().squeeze(-1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                y_true_val.extend(labels.cpu().detach().numpy())
                y_scores_val.extend(outputs_with_sigmoid.cpu().detach().numpy())

        valid_losses.append(valid_loss / len(valid_loader))
        valid_accuracies.append(100 * correct / total)
        # roc auc logic
        fpr_val, tpr_val, _ = roc_curve(y_true_val, y_scores_val)
        roc_auc_val = auc(fpr_val, tpr_val)
        roc_values_val.append(roc_auc_val)

        # Classification report
        print(f'Epoch {epoch + 1}, Train Accuracy: {train_accuracies[-1]:.2f}%, Train Loss: {train_losses[-1]:.2f}%, Train ROC-AUC: {roc_auc_train:.2f}%, Val Accuracy: {valid_accuracies[-1]:.2f}%, Val Loss: {valid_losses[-1]:.2f}%, Val ROC-AUC: {roc_auc_val:.2f}%')

        y_true_val = np.array(y_true_val)
        y_pred_val = (np.array(y_scores_val) >= 0.5).astype(int)

        # Accumulate across folds
        all_y_true.extend(y_true_val.tolist())
        all_y_scores.extend(y_scores_val)
        all_y_pred.extend(y_pred_val.tolist())
        
        print("Validation Classification Report:")
        print(classification_report(y_true_val, y_pred_val,zero_division=1))
        
         # Append metrics for current epoch and fold
        metrics[fold + 1]['train_loss'].append(train_losses[-1])
        metrics[fold + 1]['train_accuracy'].append(train_accuracies[-1])
        metrics[fold + 1]['train_roc_auc'].append(roc_values_train[-1])
        metrics[fold + 1]['valid_loss'].append(valid_losses[-1])
        metrics[fold + 1]['valid_accuracy'].append(valid_accuracies[-1])
        metrics[fold + 1]['valid_roc_auc'].append(roc_values_val[-1])

    metrics[fold + 1]['y_pred']=y_pred_val
    metrics[fold + 1]['y_true']=y_true_val

In [ ]:
# Plotting function for losses and accuracies by fold

def plot_metrics(metrics, num_folds):
    metric_names = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy', 'train_roc_auc', 'valid_roc_auc']
    
    for metric_name in metric_names:
        plt.figure(figsize=(10, 5))
        for fold in range(1, num_folds + 1):
            fold_metric = metrics[fold][metric_name]
            plt.plot(range(1, num_epochs + 1), fold_metric, label=f'Fold {fold}')
        plt.title(f'{metric_name.replace("_", " ").title()} over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric_name.replace('_', ' ').title())
        plt.legend()
        plt.show()

plot_metrics(metrics, num_folds)

**COMPLETE PATCH LEVEL RESULTS** 

In [ ]:
print("\nPatch-Level Classification Report:")
print(classification_report(all_y_true, all_y_pred, zero_division=1))

In [ ]:
conf_matrix = confusion_matrix(all_y_true, all_y_pred)
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False,
            xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Annotation Level Confusion Matrix (3-Fold CV)')
plt.show()

**AGGREGATING PATCH LEVEL RESULTS TO WSI LEVEL** 

In [ ]:
# Dictionary to store final WSI-level prediction counts across all folds
wsi_pred_counts = defaultdict(lambda: {0: 0, 1: 0})

# Loop over each fold and accumulate predictions
for fold in range(1, num_folds + 1):
    # Retrieve WSI IDs and predictions
    wsi_ids = metrics[fold]['wsi_nos']
    predicted_labels = metrics[fold]['y_pred']

    # Count 0s and 1s per WSI
    for wsi, pred in zip(wsi_ids, predicted_labels):
        pred = int(pred)  # Ensure hashable
        wsi_pred_counts[wsi][pred] += 1

wsi_pred_counts = dict(wsi_pred_counts) 
wsi_pred_counts 

In [ ]:
her2_df.head()

In [ ]:
# Lists to store WSI-level ground truths and predicted labels 

wsi_true_labels = []
wsi_pred_labels = []

# Loop over each WSI and compute majority vote
for item in wsi_pred_counts.items(): 

    if item[0] >= 901:
        print(item[0])
        true_label = tnbc_df[tnbc_df['WSI No.']== item[0]]['Label'].to_list()[0]
    elif item[0] >= 61:
        print(item[0])
        true_label = her2_df[her2_df['WSI No.']== item[0]]['Label'].to_list()[0]  
    wsi_true_labels.append(true_label)  # ground truth from external dict 
    majority_vote = 1 if item[1][1] >= item[1][0] else 0  # tie → label 1
    wsi_pred_labels.append(majority_vote) 

In [ ]:
# Classification report for test set 

print("Test WSI Level Classification Report:\n")
print(classification_report(wsi_true_labels, wsi_pred_labels)) 

In [ ]:
# Generate confusion matrix
conf_matrix = confusion_matrix(wsi_true_labels, wsi_pred_labels)

# Plot confusion matrix
plt.figure(figsize=(5, 5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('WSI Level Confusion Matrix')
plt.show()